In [ ]:
!pip install numpy datasets tiktoken wandb tqdm networkx --quiet
!pip3 install transformers --quiet
!pip install audiolm-pytorch

In [1]:
#@markdown ## Fetch and parse dataset
%cd /home/jls/repos/nanoGPT
from data.calderon.prepare import Dataset
import logging, os

# Set logging to debug
logging.basicConfig(level=logging.DEBUG)
# Example using shakespeare
ds = Dataset()
ds.load('./data/calderon/54436-0-clean.txt')
ds.parse()
ds.export('./data/calderon/')

/home/jls/Soft/nanoGPT


DEBUG:root:Dataset created with data_url: None
DEBUG:root:Parsing data with 400718 characters


train has 165036 tokens
val has 18571 tokens


In [2]:
#@markdown ## Training (custom)
"""
This training script can be run both on a single gpu in debug mode,
and also in a larger training run with distributed data parallel (ddp).

To run in debug mode example:
$ python train.py --batch_size=32 --other=args

To run DDP on 4 gpus on one node, example:
$ torchrun --standalone --nproc_per_node=4 train.py
"""

import os
import time
import math
from contextlib import nullcontext

import numpy as np
import torch
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

from model import GPTConfig, GPT

# -----------------------------------------------------------------------------
# default config values designed to train a gpt2 (124M) on OpenWebText
# I/O
out_dir = 'out-calderon'
eval_interval = 2000
log_interval = 1
eval_iters = 200
eval_only = False # if True, script exits right after the first eval
always_save_checkpoint = True # if True, always save a checkpoint after each eval
init_from = 'gpt2' # 'scratch' or 'resume' or 'gpt2*'
# wandb logging
wandb_log = False # disabled by default
wandb_project = 'calderon'
wandb_run_name = 'ft-gpt2' # 'run' + str(time.time())
# data
dataset = 'calderon'
batch_size = 12
block_size = 1024
# model
n_layer = 12
n_head = 12
n_embd = 768
dropout = 0.2 # for pretraining 0 is good, for finetuning try 0.1+
# adamw optimizer
learning_rate = 3e-5 # max learning rate
max_iters = 600000 # total number of training iterations
weight_decay = 1e-2
beta1 = 0.9
beta2 = 0.95
# learning rate decay settings
decay_lr = False # whether to decay the learning rate
warmup_iters = 2000 # how many steps to warm up for
lr_decay_iters = 600000 # should be ~= max_iters per Chinchilla
#min_lr = 6e-5 # minimum learning rate, should be ~= learning_rate/10 per Chinchilla
# DDP settings
backend = 'nccl' # 'nccl', 'gloo', etc.
# system
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1', etc.
dtype = 'float32' # 'float32' or 'bfloat16'
compile = False # use PyTorch 2.0 to compile the model to be faster

### Custom overrides
out_dir = 'out-calderon'
eval_interval = 500
wandb_log = False # feel free to turn on
wandb_project = 'calderon'
wandb_run_name = 'ft-' + str(time.time())
compile = True # takes too little time to finetune, not worth it

always_save_checkpoint = False

dataset = 'calderon'
init_from = 'gpt2'
batch_size = 1
block_size = 512

learning_rate = 1e-5
max_iters = 1000
decay_lr = False
# -----------------------------------------------------------------------------
config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
config = {k: globals()[k] for k in config_keys} # will be useful for logging
# -----------------------------------------------------------------------------
os.environ["LOCAL_RANK"] = "6"
# various inits, derived attributes, I/O setup
ddp = int(os.environ.get('LOCAL_RANK', -1)) != -1 # is this a ddp run?
if ddp:
    init_process_group(backend=backend)
    gpu_id = int(os.environ["LOCAL_RANK"])
    device = f"cuda:{gpu_id}"
else:
    gpu_id = 0 # gpu_id 0 means this is the (single) master process, basically

if gpu_id == 0:
    os.makedirs(out_dir, exist_ok=True)
torch.manual_seed(1337 + gpu_id) # note: each worker gets a different seed
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
# note: float16 would require us to change the code to use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

# poor man's data loader, TODO evaluate need for actual DataLoader
data_dir = os.path.join('data', dataset)
train_data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
val_data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint16, mode='r')
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

# init these up here, can override if init_from='resume' (i.e. from a checkpoint)
iter_num = 0
best_val_loss = 1e9

# model init. TODO: fix bug we should also propagate the correct vocab_size to the model_args
model_args = dict(n_layer = n_layer, n_head = n_head, n_embd = n_embd, block_size = block_size, dropout = dropout)
if init_from == 'scratch':
    # init a new model from scratch
    print("Initializing a new model from scratch")
    gptconf = GPTConfig(**model_args)
    model = GPT(gptconf)
elif init_from == 'resume':
    print(f"Resuming training from {out_dir}")
    # resume training from a checkpoint.
    ckpt_path = os.path.join(out_dir, 'ckpt.pt')
    checkpoint = torch.load(ckpt_path, map_location=device)
    checkpoint_model_args = checkpoint['model_args']
    for k, v in model_args.items():
        assert checkpoint_model_args[k] == v, "for now"
        # TODO: think through how passed in params should interact with checkpoint params
    gptconf = GPTConfig(**model_args)
    model = GPT(gptconf)
    state_dict = checkpoint['model']
    # fix the keys of the state dictionary :(
    # honestly no idea how checkpoints sometimes get this prefix, have to debug more
    unwanted_prefix = '_orig_mod.'
    for k,v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
    model.load_state_dict(state_dict)
    iter_num = checkpoint['iter_num']
    best_val_loss = checkpoint['best_val_loss']
elif init_from.startswith('gpt2'):
    print(f"Initializing from OpenAI GPT-2 weights: {init_from}")
    # initialize from OpenAI GPT-2 weights
    override_args = dict(dropout=dropout)
    model = GPT.from_pretrained(init_from, override_args)
    # read off and override the GPT sizing model args from the model config
    model_args['n_layer'] = model.config.n_layer
    model_args['n_head'] = model.config.n_head
    model_args['n_embd'] = model.config.n_embd
# crop down the model block size if desired
if block_size < model.config.block_size:
    model.crop_block_size(block_size)
model.to(device)

# optimizer
optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type);#'cpu')
if init_from == 'resume':
    optimizer.load_state_dict(checkpoint['optimizer'])

# compile the model
if compile:
    print("compiling the model... (takes a ~minute)")
    unoptimized_model = model
    model = torch.compile(model) # requires PyTorch 2.0

# wrap model into DDP container
if ddp:
    model = DDP(model, device_ids=[gpu_id])

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            with ctx:
                logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# learning rate decay scheduler (cosine with warmup)
def get_lr(iter):
    # 1) linear warmup for warmup_iters steps
    if iter < warmup_iters:
        return learning_rate * iter / warmup_iters
    # 2) if iter > lr_decay_iters, return min learning rate
    if iter > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (iter - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

# logging
if wandb_log and gpu_id == 0:
    import wandb
    wandb.init(project=wandb_project, name=wandb_run_name, config=config)

# training loop
t0 = time.time()
while True:

    # determine the learning rate for this iteration
    if decay_lr:
        lr = get_lr(iter_num)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
    else:
        lr = learning_rate

    if iter_num % eval_interval == 0 and gpu_id == 0:
        losses = estimate_loss()
        print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        if wandb_log:
            wandb.log({
                "iter": iter_num,
                "train/loss": losses['train'],
                "val/loss": losses['val'],
                "lr": lr,
            })
        if losses['val'] < best_val_loss or always_save_checkpoint:
            best_val_loss = losses['val']
            raw_model = model.module if ddp else model
            if iter_num > 0:
                checkpoint = {
                    'model': raw_model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'model_args': model_args,
                    'iter_num': iter_num,
                    'best_val_loss': best_val_loss,
                    'config': config,
                }
                print(f"saving checkpoint to {out_dir}")
                torch.save(checkpoint, os.path.join(out_dir, 'ckpt.pt'))
    if iter_num == 0 and eval_only:
        break

    X, Y = get_batch('train')
    with ctx:
        logits, loss = model(X, Y)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    # TODO: gradient clipping evaluate need for
    optimizer.step()

    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if iter_num % log_interval == 0 and gpu_id == 0:
        lossf = loss.item() # loss as float. TODO CPU-GPU sync: profile, make sure not slow af
        print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms")
    iter_num += 1

    # termination conditions
    if iter_num > max_iters:
        break

if ddp:
    destroy_process_group()


Initializing from OpenAI GPT-2 weights: gpt2
loading weights from pretrained gpt: gpt2
forcing vocab_size=50257, block_size=1024, bias=True
overriding dropout rate to 0.2


/home/jls/.pyenv/versions/3.10.11/envs/nanoGPT/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443


number of parameters: 123.65M


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /gpt2/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /gpt2/resolve/main/generation_config.json HTTP/1.1" 200 0


num decayed parameter tensors: 50, with 123,925,248 parameters
num non-decayed parameter tensors: 98, with 121,344 parameters
using fused AdamW: True
compiling the model... (takes a ~minute)


[2023-05-11 12:32:28,442] torch._dynamo.eval_frame: [DEBUG] skipping __init__ /home/jls/.pyenv/versions/3.10.11/lib/python3.10/contextlib.py
[2023-05-11 12:32:28,444] torch._dynamo.eval_frame: [DEBUG] skipping __enter__ /home/jls/.pyenv/versions/3.10.11/lib/python3.10/contextlib.py
[2023-05-11 12:32:28,445] torch._dynamo.eval_frame: [DEBUG] skipping __init__ /home/jls/.pyenv/versions/3.10.11/lib/python3.10/contextlib.py
[2023-05-11 12:32:28,446] torch._dynamo.eval_frame: [DEBUG] skipping __enter__ /home/jls/.pyenv/versions/3.10.11/lib/python3.10/contextlib.py
[2023-05-11 12:32:28,447] torch._dynamo.eval_frame: [DEBUG] skipping enable_dynamic /home/jls/.pyenv/versions/3.10.11/envs/nanoGPT/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py
[2023-05-11 12:32:28,454] torch._dynamo.symbolic_convert: [INFO] Step 1: torchdynamo start tracing forward
[2023-05-11 12:32:28,455] torch._dynamo.symbolic_convert: [DEBUG] TRACE starts_line /home/jls/Soft/nanoGPT/model.py:178
[2023-05-11 12:32:2

step 0: train loss 4.3496, val loss 4.3035


[2023-05-11 12:33:11,265] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_FAST self [TensorVariable()]
[2023-05-11 12:33:11,266] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_ATTR attn [TensorVariable(), NNModuleVariable()]
[2023-05-11 12:33:11,269] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_FAST self [TensorVariable(), NNModuleVariable()]
[2023-05-11 12:33:11,271] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_ATTR ln_1 [TensorVariable(), NNModuleVariable(), NNModuleVariable()]
[2023-05-11 12:33:11,273] torch._dynamo.symbolic_convert: [DEBUG] TRACE LOAD_FAST x [TensorVariable(), NNModuleVariable(), NNModuleVariable()]
[2023-05-11 12:33:11,274] torch._dynamo.symbolic_convert: [DEBUG] TRACE CALL_FUNCTION 1 [TensorVariable(), NNModuleVariable(), NNModuleVariable(), TensorVariable()]
[2023-05-11 12:33:11,277] torch._dynamo.symbolic_convert: [DEBUG] INLINING <code object forward at 0x7fe376789000, file "/home/jls/Soft/nanoGPT/model.py", line 34> 
  35           0 LOAD_G

iter 0: loss 4.7659, time 108881.69ms
iter 1: loss 5.6204, time 114.15ms
iter 2: loss 4.8298, time 157.86ms
iter 3: loss 5.1755, time 138.46ms
iter 4: loss 5.0482, time 136.74ms
iter 5: loss 4.8515, time 153.80ms
iter 6: loss 4.8296, time 144.41ms
iter 7: loss 4.7301, time 141.71ms
iter 8: loss 4.7736, time 141.44ms
iter 9: loss 4.1424, time 146.22ms
iter 10: loss 4.2852, time 140.91ms
iter 11: loss 4.6730, time 144.84ms
iter 12: loss 3.8106, time 141.64ms
iter 13: loss 4.1139, time 141.70ms
iter 14: loss 4.4279, time 142.02ms
iter 15: loss 4.5959, time 142.92ms
iter 16: loss 4.4862, time 142.43ms
iter 17: loss 4.3012, time 145.80ms
iter 18: loss 4.2965, time 141.45ms
iter 19: loss 3.6305, time 140.90ms
iter 20: loss 4.1029, time 142.77ms
iter 21: loss 4.7699, time 142.32ms
iter 22: loss 4.7316, time 141.07ms
iter 23: loss 4.7945, time 141.30ms
iter 24: loss 4.7031, time 144.99ms
iter 25: loss 4.6681, time 146.18ms
iter 26: loss 4.5303, time 141.57ms
iter 27: loss 4.4913, time 142.19ms

In [16]:
#!python sample.py --out_dir=out-calderon --device=cpu
!python sample.py \
    --init_from=resume \
    --start="Qué dijo Segismundo?" \
    --num_samples=1 --max_new_tokens=100

Overriding: init_from = gpt2
Overriding: start = Qué dijo Segismundo?
Overriding: num_samples = 1
Overriding: max_new_tokens = 100
loading weights from pretrained gpt: gpt2
forcing vocab_size=50257, block_size=1024, bias=True
overriding dropout rate to 0.0
number of parameters: 123.65M
No meta.pkl found, assuming GPT-2 encodings...
Qué dijo Segismundo?

- It has been reported that two persons have been arrested in connection with the 'accomplishments with funds related to a 'mass kidnapping' in the 'Mexico City area' in 2007.

- A man was arrested yesterday on suspicion of illegally transporting narcotics before a police operation.

- The suspect, who has been identified as Carlos 'El' Hernández, a Colombian citizen, was reportedly deported in 2008.

- The 'mass' kidnapping affair
---------------
